# Python 201.2

## Nível intermediário em Python

Os notebooks dessa segunda etapa, focam especificamente em features intermediárias/avançadas da linguagem.

Tenha em mente que algumas questões apresentadas neste notebook, farão referência aos arquivos .py encontrados dentro do diretório src no mesmo nível.

### Iterators

Iteradores em python estão por toda parte e são muito utilizados, durante este treinamento já utilizamos vários deles (como listas, tuplas e etc).

*"Em programação de computadores, um iterador se refere tanto ao objeto que permite ao programador percorrer um container, (uma coleção de elementos) particularmente listas, quanto ao padrão de projetos Iterator, no qual um iterador é usado para percorrer um container e acessar seus elementos. O padrão Iterator desacopla os algoritmos dos recipientes, porém em alguns casos, os algoritmos são necessariamente específicos dos containers e, portanto, não podem ser desacoplados."* [wiki](https://pt.wikipedia.org/wiki/Iterador)

Apesar de podermos utilizar as listas, podemos também de forma explícita percorrer as mesmas.

In [48]:
alphabet = iter('abc')
while True:
    try:
        print('it: ', next(alphabet))
    except StopIteration as e:
        print('Exception : StopIteration')
        break

it:  a
it:  b
it:  c
Exception : StopIteration


Sabendo disso, podemos permitir que uma classe sofra iteração.

In [190]:
# Implementação de Fibonacci baseada na Abordagem Iterativa
# https://pt.wikipedia.org/wiki/Sequ%C3%AAncia_de_Fibonacci#Abordagem_iterativa

class Fibonacci:
    def __init__(self, number):
        self.start(number)
    
    def start(self, number):
        self.number = number + 1
        self.__index = 0
        self.__j = 1
        self.__i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        # Fibonacci
        t = self.__index
        # from 2 to number + 1
        if 1 < self.__index <= self.number:
            t = self.__i + self.__j
            self.__i = self.__j
            self.__j = t
        # Iterator
        if self.__index == self.number:
            raise StopIteration
        self.__index += 1
        return t

# 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144,233, 377, 610, 987
# 1597, 2584, 4181, 6765, 10946

fib = Fibonacci(21)
nums = []
for i in fib:
    nums.append(i)
print('Fibonacci Sequence: ')
print(nums)

print('-' * 20)
print('Restart and iterate over while and next:')
fib.start(8)
i = 0
while True:
    try:
        print(f'num {i}: ', next(fib))
        i += 1
    except StopIteration as si:
        print('StopIteration : exception')
        break

Fibonacci Sequence: 
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
--------------------
Restart and iterate over while and next:
num 0:  0
num 1:  1
num 2:  1
num 3:  2
num 4:  3
num 5:  5
num 6:  8
num 7:  13
num 8:  21
StopIteration : exception


### [Generators](https://docs.python.org/3/tutorial/classes.html#generators)

Generators são ferramentas simples que nos permitem criar novos iteradores!

Para utilizá-los devemos utilizar a palavras reservada **yield**, acoplando isso dentro de uma função.

Os métodos **__iter__** e **__next__** são criados automaticamente quando utilizamos a criação de geradores.

E um grande diferencial, é que eles mantém o estado de execução.

In [227]:
# gerado simples
def simple_gen(x):
    for m in range(x):
        yield m

for k in simple_gen(10):
    print(k)

0
1
2
3
4
5
6
7
8
9


### Programação Funcional

Apesar do Python suportar algum tipo de programação funcional, originalmente a linguagem não foi construída para este tipo de estrutura, como eu caso de Haskell, Elm, Elixir.

Entretanto, funções em Python são First-Class Citizen, existe Closure e algumas outras funções com aspecto funcional, entretanto, não existe necessariamente um paradigma funcional completo na linguagem.

Mas vamos explorar melhor os aspectos funcionais da linguagem.

#### map, reduce, filter e lamda

Dentre as funções voltadas para o paradigma funcional temos map, reduce e filter.

 - **map**: Aplica uma função a cada elemento de uma sequência.
 - **reduce**: Aplica uma função a cada elemento de uma sequência e agrega a um total.
 - **filter**: Realiza um filtro de cada elemento dentro da sequência.

Outro paradigma é a utilização de lambda expression, que é nada mais nada menos que funções anônimas de escopo restrito que aceitam argumentos e suportam apenas uma expressão.

In [11]:
# Exemplo de map
lista = [1, 2, 3, 4, 5, 6, 7]

def sqrt(x):
    return x*x

print('map : sqrt : ', list(map(sqrt, lista)))

map : sqrt :  [1, 4, 9, 16, 25, 36, 49]


In [10]:
# Exemplo de reduce
from functools import reduce

def add(x, y):
    return x + y

# (1 + 2) + (3 + 4) + (5 + 6) + 7 = 28
print('reduce = 28 : is ok? => ', reduce(add, lista) == 28)
# (1 + 2) + (3 + 4) = 10
print('reduce = 10 : is ok? => ', reduce(add, [1, 2, 3, 4]) == 10)

reduce = 28 : is ok? =>  True
reduce = 10 : is ok? =>  True


In [24]:
# Exemplo de filter
lista = [None, 1, 2, 3, None, 4]

def gt(x):
    """greater than."""
    if x:
        return x > 2
    return False

print('filter, remover None: ', list(filter(None, lista)))
print('filter, gt > 2 only:', list(filter(gt, lista)))

filter, remover None:  [1, 2, 3, 4]
filter, gt > 2 only: [3, 4]


Claro, que ao invés de definirmos expressões tão curtas no formato de funções, podemos na verdade definir expressões lambdas para isso.

In [31]:
print('sqrt: ', list(map(lambda x : x*x, [1, 2, 3, 4, 5])))

k = [[1, 2], [3, 4]]
# Colocando a função, gerada por lambda em uma variável!
sum_up = lambda x : x[0] + x[1]
r = list(map(sum_up, k))
print(f'Soma de {k}: ', r)

sqrt:  [1, 4, 9, 16, 25]
Soma de [[1, 2], [3, 4]]:  [3, 7]


#### Closure

Closure refere-se a capacidade de se criar funções dentro do escopo de outras funções encapsulando assim determinado comportamento de modo interno apenas.

Na prática Closure funciona quase como uma representação mais simples de um objeto. Em javascript nos meados da internet, essa definição de Closures foi muito utilizada (e ainda é muito ainda hoje, principalmente nos frameworks).

Closure são bem importantes, pois através desse paradigma podemos definir comportamentos a nossas funções de modo padronizado, e closure é um primeiro passo para o entendimento de decoradores.


In [39]:
def master(x):
    y = 2
    def slave():
        return x + y
    return slave

closure = master(5)
print('resultado: ', closure())

resultado:  7


#### DataClass

Na nova versão da linguagem (3.7+) temos a implementação da **DataClass**, o qual cria uma **decorador** (que veremos mais a frente como utilizar), para anotar a classe e adicionar diversos métodos já implementados nela.